# Otimização de Supply Chain

Este exemplo demonstra (1) análise de desempenho de fornecedores (On-Time Delivery, Lead Time, Taxa de Defeitos, Custo) e (2) otimização de estoque via política baseada em nível de serviço. A ideia é identificar fornecedores ideais e acionar ajustes de reposição para reduzir rupturas e custos totais.

## Métricas-chave
- On-Time Delivery (OTD): % de pedidos entregues no prazo.
- Lead Time médio e sua variabilidade.
- Defect Rate: % de itens com defeito.
- Unit Cost e Custo total ponderado.

Com base nessas métricas, ranqueamos fornecedores e sugerimos um portfólio ótimo (p.ex., manter múltiplas fontes para mitigar risco).

In [ ]:
# (2) Leitura dos dados
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('default')
sns.set_palette('husl')

# Espera-se um arquivo notebooks/operations_data.csv com colunas, por exemplo:
# supplier, on_time_rate, lead_time_days, defect_rate, unit_cost, quantity
df = pd.read_csv('notebooks/operations_data.csv')
df.head()

In [ ]:
# (3) Análise de métricas de entrega e sugestão de fornecedores ideais
metrics = df.copy()
# Normalizações/penalidades para compor um score: maior melhor
# Pesos podem ser ajustados conforme estratégia
w_otd, w_lt, w_def, w_cost = 0.35, 0.25, 0.2, 0.2
# Escalas (min-max) para lead time, defeito e custo (quanto menor melhor)
for col in ['lead_time_days','defect_rate','unit_cost']:
    cmin, cmax = metrics[col].min(), metrics[col].max()
    metrics[f'{col}_norm'] = 1 - (metrics[col] - cmin) / (cmax - cmin + 1e-9)
# OTD já é % alto melhor; normalizamos a 0-1 se necessário
if metrics['on_time_rate'].max() > 1.0:
    metrics['otd_norm'] = metrics['on_time_rate'] / 100.0
else:
    metrics['otd_norm'] = metrics['on_time_rate']

metrics['supplier_score'] = (w_otd*metrics['otd_norm'] +
                             w_lt*metrics['lead_time_days_norm'] +
                             w_def*metrics['defect_rate_norm'] +
                             w_cost*metrics['unit_cost_norm'])

ranking = metrics.sort_values('supplier_score', ascending=False)
top_suppliers = ranking[['supplier','supplier_score','on_time_rate','lead_time_days','defect_rate','unit_cost']].head(5)
print('Top fornecedores sugeridos:')
display(top_suppliers)

# Exemplo simples de alocação sugerida baseada em score (proporcional)
scores = ranking.set_index('supplier')['supplier_score']
alloc = (scores / scores.sum()).round(2)
print('Sugestão de alocação proporcional ao desempenho:')
display(alloc.to_frame('allocation_share'))


In [ ]:
# (4) Gráfico de barras de desempenho dos fornecedores
fig, ax = plt.subplots(figsize=(10,5))
order = metrics.sort_values('supplier_score', ascending=False)['supplier']
sns.barplot(data=metrics, x='supplier', y='supplier_score', order=order, ax=ax)
ax.set_title('Score de Desempenho por Fornecedor')
ax.set_xlabel('Fornecedor')
ax.set_ylabel('Score (0-1)')
plt.xticks(rotation=30, ha='right')
plt.tight_layout()
plt.show()

# Interpretação e Recomendações
- Fornecedores com maior score combinam alta pontualidade, menor lead time, baixa taxa de defeitos e custo competitivo.
- Recomenda-se concentrar pedidos nos top fornecedores, mantendo ao menos um fornecedor secundário para resiliência.
- Ajuste de estoque: para itens com alta variabilidade de lead time ou baixa OTD, elevar estoque de segurança; para itens estáveis, reduzir.
- Revise pesos (w_otd, w_lt, w_def, w_cost) conforme estratégia (p.ex., priorizar qualidade vs. custo).
- Monitore mensalmente e reestime scores para capturar mudanças de desempenho.